In [0]:
use catalog `price-prediction-analytics`;

Create or replace table silver.reporting_dim_state_Stage_1 AS

select
distinct state_name
from silver.daily_pricing
WHERE source_file_load_date > (SELECT nvl(max(PROCESSED_TABLE_DATETIME),'1900-01-01') FROM processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS 
WHERE process_name = 'reportingDimensionTablesLoad' AND process_status = 'Completed' )

In [0]:
create or replace table silver.reporting_dim_state_stage_2 AS
select 
    silverDim.state_name, 
    row_number() over (order by silverDim.state_name) AS STATE_ID,
    current_timestamp() as created_at, 
    current_timestamp() as updated_at
from 
    silver.reporting_dim_state_Stage_1 as silverDim
left outer join 
    gold.reporting_dim_state_gold as goldDim
on 
    silverDim.state_name = goldDim.state_name
where 
    goldDim.state_name is null

In [0]:
create or replace table `price-prediction-analytics`.silver.reporting_dim_state_stage_3 AS
select 
silverDim.state_name, silverDim.state_id + Prev_max_SK_ID AS STATE_ID,
current_timestamp() as lakehouse_inserted_date, 
current_timestamp() as lakehouse_updated_date
from silver.reporting_dim_state_stage_2 silverDim
Cross join (Select NVL(max(State_id),0) as Prev_max_SK_ID from gold.reporting_dim_state_gold) goldDim

In [0]:
INSERT INTO gold.reporting_dim_state_gold
SELECT state_name,
state_id,
current_timestamp(),
current_timestamp()
FROM silver.reporting_dim_state_stage_3

In [0]:
select * from gold.reporting_dim_state_gold